In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import preprocessing
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
import sklearn

#evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

#nomoralization
from sklearn.preprocessing import StandardScaler

#cross validation
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold

#up balance
from sklearn.utils import resample



/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# I have to define our own f1 socre funciton, sicne Xgboost does not have this evaluation metric

In [3]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_true, y_hat):
    y_true = y_true
    y_hat = [1. if y_cont > 0.5 else 0. for y_cont in y_hat] # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True
# I have to define our own f1 socre funciton, sicne Xgboost does not have this evaluation metric
def evalerror_f1(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result. The metric name must not contain a colon (:) or a space
    # since preds are margin(before logistic transformation, cutoff at 0)
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in preds]
    #print(labels)
    #print(preds)
    #print(int(1))
    return 'f1-score', f1_score(labels, y_bin)

In [4]:
def trainer(df, classifier, fill_na=False):
    if fill_na == True:
        df = df.fillna(-100)
    
    #preparing with kFold =5 
    kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
    
    y_pred_all=[]
    y_test_all=[]
    
    #splitting tran and test
    for train, test in kf.split(df):
        df_train =  df.iloc[train]
        df_test =  df.iloc[test]

        # Separate majority and minority classes    
        df_train_major = df_train[df_train.Issue_YN==0]
        df_trian_minor = df_train[df_train.Issue_YN==1]

        # Upsample minority class
        df_minor_upsampled = resample(df_trian_minor, 
                                         replace=True,                     # sample with replacement
                                         n_samples=len(df_train_major),    # to match majority class
                                         random_state=123)                 # reproducible results

        df_train = pd.concat([df_train_major, df_minor_upsampled]).copy()
        # Display new class counts
        #print(df_train.Issue_YN.value_counts())
        
        #Separate X and Y
        X_train = df_train.drop(columns=['Issue_YN', 'Pseudo_ClientID'], axis =1).copy()
        y_train = df_train['Issue_YN']
        X_test = df_test.drop(columns=['Issue_YN', 'Pseudo_ClientID'], axis =1).copy()
        y_test = df_test['Issue_YN']
        
        # Fit the model, NEED TO BE MODIFIED DEPEND ON MODELS
        classifier.fit(X_train, y_train,
                       eval_set=[(X_test, y_test)],
                       eval_metric=evalerror_f1,
                       early_stopping_rounds=300,
                      verbose=200)
        
        # evaluate the model, NEED TO BE MODIFIED DEPEND ON MODELS
        y_pred=classifier.predict_proba(X_test)[:,1]

        
        y_pred_all.append(y_pred)
        y_test_all.append(y_test)
        
    return np.array(y_pred_all), np.array(y_test_all)

# EXAMPLE

In [5]:
df=pd.read_csv('./df_all.csv')
features = pd.read_csv('./important0.csv')
cols=features['Features'].tolist()
df=df.drop(columns=cols)
df=df.drop(['Pseudo_ClientID@'],axis=1)

In [6]:
df.head()

,Pseudo_ClientID,ACTV_WRKR_COUNT,WRKR_COUNT,BANK_COUNT,CHECK_COUNT,GL_ACCT_COUNT,LOCATION_COUNT,ORG_UNIT_COUNT,PAYRUN_COUNT,PAY_PERIOD_COUNT,POSITION_COUNT,NULL_COUNT,ACTV_JOB_COUNT-ACTV_WRKR_COUNT,ACTV_JOB_COUNT+ACTV_WRKR_COUNT,ACTV_JOB_COUNT+WRKR_COUNT,ACTV_JOB_COUNT+BANK_COUNT,ACTV_JOB_COUNTxCHECK_COUNT,ACTV_JOB_COUNT-CLT_PAYCOMP_COUNT,ACTV_JOB_COUNT+CLT_PAYCOMP_COUNT,ACTV_JOB_COUNTxCLT_PAYCOMP_COUNT,ACTV_JOB_COUNT-CLT_TAX_COUNT,ACTV_JOB_COUNT+CLT_TAX_COUNT,ACTV_JOB_COUNTxCLT_TAX_COUNT,ACTV_JOB_COUNT-GL_ACCT_COUNT,ACTV_JOB_COUNT+GL_ACCT_COUNT,ACTV_JOB_COUNTxGL_ACCT_COUNT,ACTV_JOB_COUNT-JOB_COUNT,ACTV_JOB_COUNT+LOCATION_COUNT,ACTV_JOB_COUNTxLOCATION_COUNT,ACTV_JOB_COUNT-ORG_UNIT_COUNT,ACTV_JOB_COUNT+ORG_UNIT_COUNT,ACTV_JOB_COUNTxORG_UNIT_COUNT,ACTV_JOB_COUNTxPAY_PERIOD_COUNT,ACTV_JOB_COUNT+WRKR_PAY_COMP_AVG,ACTV_JOB_COUNTxWRKR_PAY_COMP_AVG,ACTV_WRKR_COUNT-WRKR_COUNT,ACTV_WRKR_COUNT+WRKR_COUNT,ACTV_WRKR_COUNTxWRKR_COUNT,ACTV_WRKR_COUNT-BANK_COUNT,ACTV_WRKR_COUNT+BANK_COUNT,ACTV_WRKR_COUNTxBANK_COUNT,ACTV_WRKR_COUNT-CHECK_COUNT,ACTV_WRKR_COUNT+CHECK_COUNT,ACTV_WRKR_COUNTxCHECK_COUNT,ACTV_WRKR_COUNT-CLT_PAYCOMP_COUNT,ACTV_WRKR_COUNT+CLT_PAYCOMP_COUNT,ACTV_WRKR_COUNTxCLT_PAYCOMP_COUNT,ACTV_WRKR_COUNT-CLT_TAX_COUNT,ACTV_WRKR_COUNT+CLT_TAX_COUNT,ACTV_WRKR_COUNTxCLT_TAX_COUNT,ACTV_WRKR_COUNT-GL_ACCT_COUNT,ACTV_WRKR_COUNT+GL_ACCT_COUNT,ACTV_WRKR_COUNTxGL_ACCT_COUNT,ACTV_WRKR_COUNT-JOB_COUNT,ACTV_WRKR_COUNT+JOB_COUNT,ACTV_WRKR_COUNT-LOCATION_COUNT,ACTV_WRKR_COUNT+LOCATION_COUNT,ACTV_WRKR_COUNTxLOCATION_COUNT,ACTV_WRKR_COUNT-ORG_UNIT_COUNT,ACTV_WRKR_COUNT+ORG_UNIT_COUNT,ACTV_WRKR_COUNTxORG_UNIT_COUNT,ACTV_WRKR_COUNT-PAYRUN_COUNT,ACTV_WRKR_COUNT+PAYRUN_COUNT,ACTV_WRKR_COUNTxPAYRUN_COUNT,ACTV_WRKR_COUNT-PAY_PERIOD_COUNT,ACTV_WRKR_COUNT+PAY_PERIOD_COUNT,ACTV_WRKR_COUNTxPAY_PERIOD_COUNT,ACTV_WRKR_COUNT-POSITION_COUNT,ACTV_WRKR_COUNT+POSITION_COUNT,ACTV_WRKR_COUNTxPOSITION_COUNT,ACTV_WRKR_COUNT-WRKR_PAY_COMP_AVG,ACTV_WRKR_COUNT+WRKR_PAY_COMP_AVG,ACTV_WRKR_COUNTxWRKR_PAY_COMP_AVG,ACTV_WRKR_COUNT-WRKR_TAX_AVG,ACTV_WRKR_COUNT+WRKR_TAX_AVG,ACTV_WRKR_COUNTxWRKR_TAX_AVG,WRKR_COUNT-BANK_COUNT,WRKR_COUNT+BANK_COUNT,WRKR_COUNTxBANK_COUNT,WRKR_COUNT-CHECK_COUNT,WRKR_COUNT+CHECK_COUNT,WRKR_COUNTxCHECK_COUNT,WRKR_COUNT-CLT_PAYCOMP_COUNT,WRKR_COUNT+CLT_PAYCOMP_COUNT,WRKR_COUNTxCLT_PAYCOMP_COUNT,WRKR_COUNT-CLT_TAX_COUNT,WRKR_COUNT+CLT_TAX_COUNT,WRKR_COUNTxCLT_TAX_COUNT,WRKR_COUNT-GL_ACCT_COUNT,WRKR_COUNT+GL_ACCT_COUNT,WRKR_COUNTxGL_ACCT_COUNT,WRKR_COUNT-JOB_COUNT,WRKR_COUNT+JOB_COUNT,WRKR_COUNTxJOB_COUNT,WRKR_COUNT-LOCATION_COUNT,WRKR_COUNT+LOCATION_COUNT,WRKR_COUNTxLOCATION_COUNT,WRKR_COUNT-ORG_UNIT_COUNT,WRKR_COUNT+ORG_UNIT_COUNT,WRKR_COUNTxORG_UNIT_COUNT,WRKR_COUNT-PAYRUN_COUNT,WRKR_COUNT+PAYRUN_COUNT,WRKR_COUNTxPAYRUN_COUNT,WRKR_COUNT-PAY_PERIOD_COUNT,WRKR_COUNT+PAY_PERIOD_COUNT,WRKR_COUNTxPAY_PERIOD_COUNT,WRKR_COUNT-POSITION_COUNT,WRKR_COUNT+POSITION_COUNT,WRKR_COUNTxPOSITION_COUNT,WRKR_COUNT-WRKR_PAY_COMP_AVG,WRKR_COUNT+WRKR_PAY_COMP_AVG,WRKR_COUNTxWRKR_PAY_COMP_AVG,WRKR_COUNT-WRKR_TAX_AVG,WRKR_COUNT+WRKR_TAX_AVG,WRKR_COUNTxWRKR_TAX_AVG,BANK_COUNT-CHECK_COUNT,BANK_COUNT+CHECK_COUNT,BANK_COUNTxCHECK_COUNT,BANK_COUNT-CLT_PAYCOMP_COUNT,BANK_COUNT+CLT_PAYCOMP_COUNT,BANK_COUNTxCLT_PAYCOMP_COUNT,BANK_COUNT-CLT_TAX_COUNT,BANK_COUNT+CLT_TAX_COUNT,BANK_COUNTxCLT_TAX_COUNT,BANK_COUNT-GL_ACCT_COUNT,BANK_COUNT+GL_ACCT_COUNT,BANK_COUNTxGL_ACCT_COUNT,BANK_COUNT+JOB_COUNT,BANK_COUNTxJOB_COUNT,BANK_COUNT-LOCATION_COUNT,BANK_COUNT+LOCATION_COUNT,BANK_COUNTxLOCATION_COUNT,BANK_COUNT-ORG_UNIT_COUNT,BANK_COUNT+ORG_UNIT_COUNT,BANK_COUNTxORG_UNIT_COUNT,BANK_COUNT-PAYRUN_COUNT,BANK_COUNT+PAYRUN_COUNT,BANK_COUNTxPAYRUN_COUNT,BANK_COUNT-PAY_PERIOD_COUNT,BANK_COUNT+PAY_PERIOD_COUNT,BANK_COUNTxPAY_PERIOD_COUNT,BANK_COUNT-POSITION_COUNT,BANK_COUNT+POSITION_COUNT,BANK_COUNTxPOSITION_COUNT,BANK_COUNTxWRKR_PAY_COMP_AVG,BANK_COUNT-WRKR_TAX_AVG,BANK_COUNTxWRKR_TAX_AVG,CHECK_COUNT-CLT_PAYCOMP_COUNT,CHECK_COUNT+CLT_PAYCOMP_COUNT,CHECK_COUNTxCLT_PAYCOMP_COUNT,CHECK_COUNT-CLT_TAX_COU

In [7]:
clf = XGBClassifier(n_estimators=1000,max_depth=4,colsample_bytree=0.5,
                    gamma=0.9,min_child_weight=40,reg_alpha=0.1,reg_lambda=0.5,subsample = 0.8
                    ,n_jobs=22)

In [8]:
pred_all,test_all = trainer(df, clf)
pred=np.zeros((5,7503))
pred[pred_all>5] = 1

for i in range(5):
    print('confusion matrix:', confusion_matrix(test_all[i],pred[i]))
    print('f1 score:', f1_score(test_all[i],pred[i]))
    print('accuracy score:', accuracy_score(test_all[i],pred[i]))
    print('recall score:', recall_score(test_all[i],pred[i]))
    print()

[0]	validation_0-error:0.147274	validation_0-f1-score:0.399783
Multiple eval metrics have been passed: 'validation_0-f1-score' will be used for early stopping.

Will train until validation_0-f1-score hasn't improved in 300 rounds.
[200]	validation_0-error:0.079835	validation_0-f1-score:0.541699
Stopping. Best iteration:
[0]	validation_0-error:0.147274	validation_0-f1-score:0.399783

[0]	validation_0-error:0.117286	validation_0-f1-score:0.450687
Multiple eval metrics have been passed: 'validation_0-f1-score' will be used for early stopping.

Will train until validation_0-f1-score hasn't improved in 300 rounds.
[200]	validation_0-error:0.083167	validation_0-f1-score:0.522936
Stopping. Best iteration:
[2]	validation_0-error:0.120885	validation_0-f1-score:0.446614

[0]	validation_0-error:0.147541	validation_0-f1-score:0.415214
Multiple eval metrics have been passed: 'validation_0-f1-score' will be used for early stopping.

Will train until validation_0-f1-score hasn't improved in 300 round

/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/software/anaconda3/5.2.0/lib/python3.6/site-pac

In [9]:
pred=np.zeros((5,7503))
pred[pred_all>0.5] = 1

for i in range(5):
    print('confusion matrix:', confusion_matrix(test_all[i],pred[i]))
    print('f1 score:', f1_score(test_all[i],pred[i]))
    print('accuracy score:', accuracy_score(test_all[i],pred[i]))
    print('recall score:', recall_score(test_all[i],pred[i]))
    print()

confusion matrix: [[6624  463]
 [  74  342]]
f1 score: 0.5601965601965603
accuracy score: 0.9284286285485805
recall score: 0.8221153846153846

confusion matrix: [[6608  495]
 [  65  335]]
f1 score: 0.5447154471544715
accuracy score: 0.9253631880581101
recall score: 0.8375

confusion matrix: [[6633  447]
 [  73  350]]
f1 score: 0.5737704918032787
accuracy score: 0.9306943889111022
recall score: 0.8274231678486997

confusion matrix: [[6624  452]
 [  78  349]]
f1 score: 0.5684039087947883
accuracy score: 0.9293615886978542
recall score: 0.8173302107728337

confusion matrix: [[6655  440]
 [  63  345]]
f1 score: 0.5783738474434199
accuracy score: 0.9329601492736239
recall score: 0.8455882352941176



In [11]:
result = []
for i in pred_all:
    result.extend(i)

In [12]:
len(result)

37515

In [13]:
pred_all

array([[1.0085746e-03, 4.2421859e-02, 1.4657933e-01, ..., 3.1245491e-04,
        2.7980337e-02, 3.9676143e-04],
       [6.7021204e-03, 1.5379567e-02, 1.2209849e-03, ..., 3.8899162e-03,
        3.9121024e-03, 1.7258743e-02],
       [8.8067463e-04, 1.5842053e-01, 4.2815795e-03, ..., 3.0598440e-03,
        8.2707768e-03, 1.2261530e-03],
       [4.3372080e-01, 1.6338611e-02, 3.1312838e-01, ..., 1.3719550e-03,
        8.6569422e-01, 1.4747018e-02],
       [7.2983705e-04, 2.1096386e-03, 1.4955571e-03, ..., 7.9868920e-03,
        1.2025229e-03, 2.1860164e-03]], dtype=float32)

In [14]:
result_df = pd.DataFrame({'xgboost_out':result})

# output from xgboost model is going to be blended with other models outputs

In [16]:
result_df.to_csv('Xgboost_out.csv',index=False)

In [26]:
clf = XGBClassifier(n_estimators=1000,max_depth=4,colsample_bytree=0.5,
                    gamma=0.9,min_child_weight=40,reg_alpha=0.1,reg_lambda=0.5,subsample = 0.8
                    ,n_jobs=22)

In [27]:
pred_all = trainer(df, clf)[0]
test_all = trainer(df, clf)[1]

for i in range(5):
    print('confusion matrix:', confusion_matrix(pred_all[i],test_all[i]))
    print('f1 score:', f1_score(pred_all[i],test_all[i]))
    print('accuracy score:', accuracy_score(pred_all[i],test_all[i]))
    print('recall score:', recall_score(pred_all[i],test_all[i]))
    print()

/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empt

confusion matrix: [[6740  105]
 [ 347  311]]
f1 score: 0.5791433891992551
accuracy score: 0.9397574303611889
recall score: 0.4726443768996961

confusion matrix: [[6753  105]
 [ 350  295]]
f1 score: 0.5645933014354068
accuracy score: 0.9393575902972144
recall score: 0.4573643410852713

confusion matrix: [[6753  105]
 [ 327  318]]
f1 score: 0.595505617977528
accuracy score: 0.942423030787685
recall score: 0.4930232558139535

confusion matrix: [[6773  101]
 [ 303  326]]
f1 score: 0.6174242424242423
accuracy score: 0.9461548713847794
recall score: 0.5182829888712241

confusion matrix: [[6784   94]
 [ 311  314]]
f1 score: 0.6079380445304936
accuracy score: 0.9460215913634547
recall score: 0.5024



/software/anaconda3/5.2.0/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [57]:
pred_all=np.array(pred_all)

In [58]:
pred_all[pred_all==1]

array([1, 1, 1, ..., 1, 1, 1])

In [60]:
pred=np.zeros((5,7503))

pred[pred_all==1] = 0.5

In [65]:
pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [71]:
pred_all

array([[9.68584550e-01, 3.14154497e-02],
       [9.88928395e-01, 1.10716053e-02],
       [8.73852312e-01, 1.26147688e-01],
       [9.94337810e-01, 5.66219040e-03],
       [9.99414345e-01, 5.85655261e-04]])